In [2]:
# !pip install -q ipython-autotime
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 0 ns (started: 2023-08-30 10:23:19 +03:00)


In [3]:
import requests, os, tarfile
from tqdm import tqdm
from data_download import download, extract_tgz

dataset_url = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
segmentations = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102segmentations.tgz'
chi_distance = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/distancematrices102.mat'
labels = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat'
splits = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat'

OVERWRITE = False

download(dataset_url, dataset_url.split('/')[-1], force_redownload=OVERWRITE)
# download(segmentations, segmentations.split('/')[-1], force_redownload=OVERWRITE)
# download(chi_distance, chi_distance.split('/')[-1], force_redownload=OVERWRITE)
download(labels, labels.split('/')[-1], force_redownload=OVERWRITE)
download(splits, splits.split('/')[-1], force_redownload=OVERWRITE)
extract_tgz(r'102flowers.tgz')

File already exists
File already exists
File already exists
File already extracted
time: 16 ms (started: 2023-08-30 10:23:19 +03:00)


In [4]:
import torch
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from scipy.io import loadmat
import numpy as np
import pandas as pd
from data_processing import prepare_df, FlowerDataset


split_path = r"data\setid.mat"
labels_Path = r"data\imagelabels.mat"
data_root = r"data\102flowers\jpg"
train_split, test_split, val_split = prepare_df(split_path, labels_Path, data_root)

transformsations = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

train_dataset = FlowerDataset(train_split, transform=transformsations)
test_dataset = FlowerDataset(test_split, transform=transformsations)
val_dataset = FlowerDataset(val_split, transform=transformsations)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
train_split

,img_id,set_type,labels,image_path
0,1,0,77,data\102flowers\jpg\image_00001.jpg
1,2,0,77,data\102flowers\jpg\image_00002.jpg
2,3,0,77,data\102flowers\jpg\image_00003.jpg
3,4,0,77,data\102flowers\jpg\image_00004.jpg
4,5,0,77,data\102flowers\jpg\image_00005.jpg
...,...,...,...,...
6144,8183,0,62,data\102flowers\jpg\image_08183.jpg
6145,8184,0,62,data\102flowers\jpg\image_08184.jpg
6146,8186,0,62,data\102flowers\jpg\image_08186.jpg
6147,8188,0,62,data\102flowers\jpg\image_08188.jpg


time: 609 ms (started: 2023-08-30 10:23:19 +03:00)


In [5]:
import torchvision
import torchsummary
import torch.nn as nn
import segmentation_models_pytorch as smp

from models  import Resnet50Flower102
device = "cuda" if torch.cuda.is_available() else "cpu"
    
model = Resnet50Flower102(pretrained=True).to(device)
# torchsummary.summary(pretrained_resnet50, input_size=(3, 224, 224), device=device)

c:\Users\DELL\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\DELL\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


time: 594 ms (started: 2023-08-30 10:23:20 +03:00)


In [9]:
import torch
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
epochs = 5

for epoch in range(epochs):
    Epoch_loss = 0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = loss(y_pred, y)
        Epoch_loss = Epoch_loss + loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} loss is {Epoch_loss}")
        

RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 4.00 GiB total capacity; 3.24 GiB already allocated; 0 bytes free; 3.41 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

time: 1.02 s (started: 2023-08-30 10:25:23 +03:00)
